In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import deeplatent as dl

if torch.cuda.is_available():
    device = torch.device("cuda:0")
# else:
#      device = torch.device("cpu")
# device = torch.device("cuda:0") 

In [2]:
# pass in column names for each CSV
u_cols = ['user_id',  'sex','age', 'occupation', 'zip_code']
users = pd.read_csv('../data/ml-1m/users.dat', sep='::', names=u_cols, nrows= 100,
                    encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('../data/ml-1m/ratings.dat', sep='::', names=r_cols,
                      encoding='latin-1')

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'genre', ]
movies = pd.read_csv('../data/ml-1m/movies.dat', sep='::', names=m_cols, usecols=range(3),encoding='latin-1')
movies['genre'] = movies.genre.str.split('|')


# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(users, movie_ratings)
lens['sex'] = lens['sex'].apply(lambda x: 1 if x == 'F' else 0)

In [3]:
lens.head()

,user_id,sex,age,occupation,zip_code,movie_id,title,genre,rating,unix_timestamp
0,1,1,1,10,48067,1,Toy Story (1995),"[Animation, Children's, Comedy]",5,978824268
1,1,1,1,10,48067,48,Pocahontas (1995),"[Animation, Children's, Musical, Romance]",5,978824351
2,1,1,1,10,48067,150,Apollo 13 (1995),[Drama],5,978301777
3,1,1,1,10,48067,260,Star Wars: Episode IV - A New Hope (1977),"[Action, Adventure, Fantasy, Sci-Fi]",4,978300760
4,1,1,1,10,48067,527,Schindler's List (1993),"[Drama, War]",5,978824195


In [8]:
y = lens.rating.values
x = lens[['user_id','movie_id','sex','age','occupation']].values

y = torch.from_numpy(y).to(torch.float64)
x = torch.from_numpy(x).to(torch.int64)

In [9]:
encode_dim = (x).shape[0]



In [10]:
model = dl.DeepLatentNN(n_u=encode_dim)

In [7]:

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
for t in range(500):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)
    
    # Compute and print loss
    loss = criterion(y_pred, y.float())
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

97],
         [3.6779],
         [3.7285],
         [3.7162]],

        [[3.7297],
         [3.6779],
         [3.7285],
         [3.7162]],

        [[3.7297],
         [3.6779],
         [3.7285],
         [3.7162]]], grad_fn=<AddBackward0>)
tensor([[[3.7357],
         [3.7357],
         [3.7357],
         [3.6642]],

        [[3.7357],
         [3.7357],
         [3.7357],
         [3.6642]],

        [[3.7357],
         [3.7357],
         [3.7357],
         [3.6642]],

        ...,

        [[3.7347],
         [3.6858],
         [3.7319],
         [3.7235]],

        [[3.7347],
         [3.6858],
         [3.7319],
         [3.7235]],

        [[3.7347],
         [3.6858],
         [3.7319],
         [3.7235]]], grad_fn=<AddBackward0>)
tensor([[[3.7344],
         [3.7344],
         [3.7344],
         [3.6665]],

        [[3.7344],
         [3.7344],
         [3.7344],
         [3.6665]],

        [[3.7344],
         [3.7344],
         [3.7344],
         [3.6665]],

        ...,

  